In [1]:
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()):
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hello again!')

2022-12-19 14:57:10.036019: Hi there!
2022-12-19 14:57:10.036019: Hello again!


### This doesn't work as expected. The timestamps are the same because datetime.now is executed only a single time: when the function is defined.

In [2]:
#The conventiona forachieving the desired result in Python is to provide a default value of None
# and to  document the actual behavior in the docstring. When your code sees the argument value None, 
# you allocate the default value accrodingly

def log(message, when=None):
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hello again!')

2022-12-19 15:02:47.349556: Hi there!
2022-12-19 15:02:47.452537: Hello again!


Using None for default argument values is especially important when the arguments are mutable.

In [4]:
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

The problem here is the same as in the datetime.now example above.
The dictionary specified for default will be shared by all calls to decode because default argument values are evaluated only once(at module load time). This can cause extremely surprising behavior

In [5]:
foo = decode('bad datat')
foo['stuff'] = 5
print('Foo:', foo)
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5}
Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}


In [6]:
assert foo is bar

In [7]:
# The fix is to set the keyword argument default value to None and then document the behavior in the function's docstring

def decode(data, default=None):
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

foo = decode('bad datat')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)
assert foo is not bar

Foo: {'stuff': 5}
Bar: {'meep': 1}


This approach also works with type annotations. Here, the when argument is marked as having an Optional value that is a datetime. Thus, the only two valid choices for when are None or datetime object

In [8]:
from typing import Optional

def log_typed(message: str,
              when: Optional[datetime]=None) -> None :
    if when is None:
        when = datetiem.now()
    print(f'{when}: {message}')